# Inference User

## Import dependencies

In [1]:
import os
from avato import Client
from avato import Secret
from avato_tflite_dynamic import TFLITEDYNAMIC_Instance

## Login to avato

In [5]:
inference_user_client = Client(
    username=os.environ["INFERENCE_USER_ID"],
    password=os.environ["INFERENCE_USER_PASSWORD"],
    instance_types=[TFLITEDYNAMIC_Instance],
    backend_host="avato-staging.westeurope.cloudapp.azure.com",
    backend_port="15005"
)

## Get TFLite instance

In [6]:
# Here the inference user would be given the instance id
# by the model owner.
INSTANCE_ID = "c72b5008bc1d2e36ca2102d5f730eccb553e7a770f3b49c4dae16499126ecd22"
inference_user_instance = inference_user_client.get_instance(
    INSTANCE_ID
)

## Check security guarantees

In [8]:
# The inference user performs the same security
# checks that the owner did
inference_user_instance.validate_fatquote(
    expected_measurement="fd424f1251f75b48a5e23e6e8c390c9f919ff250de2c1a75218b6685b23a59b4",
    accept_debug=True,
    accept_group_out_of_date=True,
)

## Creating (randomly) a public-private keypair and setting it

In [9]:
inference_user_secret = Secret()
inference_user_instance.set_secret(inference_user_secret)

## Get the model format

In [10]:
#  Some relevant fields explained in more detail:
# `modelSha256`: Provides a hash of the executed remote model. This adds some notion of identity.
#  `inputTensorFormats`: Format of the input tensors.
#  `outputTensorFormats`: Format of the output tensors.
model_format = inference_user_instance.get_model_format()
print(model_format)

modelSha256: "!\214\373(\213\203K\324r\273\256\2402a\333o~\033\\\310\342\263\201K\355\332\t\232M\025\017>"
modelSize: 408612
arenaSize: 131072
inputTensorFormats {
  dimensions: 1
  dimensions: 28
  dimensions: 28
  valueType: "float32le"
  size: 3136
}
outputTensorFormats {
  dimensions: 1
  dimensions: 10
  valueType: "float32le"
  size: 40
}



## Make a prediction

In [11]:
from tensorflow import keras

# Import the Fashion MNIST dataset.
fashion_mnist = keras.datasets.fashion_mnist
(_, _), (test_images, _) = fashion_mnist.load_data()
# Preprocess the data: Data normalization.
test_images = test_images / 255.0
test_image = test_images[0:1, :, :]


# Make a prediction using the model running in the secure enclave
prediction_remote = inference_user_instance.predict(test_image)
print(
    f"Remote - Label: {prediction_remote.argmax()} with weight: {prediction_remote.max()}"
)

Remote - Label: 9 with weight: 0.9994888305664062
